In [1]:
import pandas as pd
import numpy as np
import codecs

In [2]:
data = codecs.open('Text.txt', 'r', 'utf_8_sig').read()
data_chapter = []
cur_index = 0
i = 0
while data.find('CHAPTER', cur_index) != -1:
    i += 1
    with codecs.open('Text' + str(i)  + '.txt', 'w', 'utf_8_sig') as file:
        file.write(data[data.find('CHAPTER', cur_index): data.find('CHAPTER', data.find('CHAPTER', cur_index) + 1)])
    cur_index = data.find('CHAPTER', cur_index) + 1

In [3]:
verbs = []
def get_verbs(tag):
    if tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
        return True
    return False
def clean_text(text):
    import re
    import nltk
    from nltk.corpus import stopwords
    from nltk.stem import PorterStemmer, WordNetLemmatizer
    words = nltk.pos_tag(nltk.word_tokenize(text))
    for word in words:
        if get_verbs(word[1]) and len(word[0])>2:
            verbs.append(word[0])
    text = re.sub(r"[^\w\s]", "", text, re.UNICODE)
    
    text = text.lower()
    
    lemmatizer = WordNetLemmatizer()
    
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = ['be' if obj in ['im']  else obj for obj in text]
    stop_words = stopwords.words("english")
    stop_words += ['oh']
    text = [word for word in text if not word in stop_words]
    
    text = " ".join(text)
    return text

def all_text(count = 12):
    from sklearn.feature_extraction.text import TfidfVectorizer
    for i in range(1, count + 1):
        data = pd.read_csv('Text' + str(i) + '.txt', delimiter="\t")
        
        data.iloc[:,0] = data.iloc[:,0].apply(lambda x: clean_text(x))
        
        X = data.iloc[:,0].tolist()
    
        vectorizer_tftidf = TfidfVectorizer()
        
        X = vectorizer_tftidf.fit_transform(X)
        idx = np.ravel(X.sum(axis=0).argsort(axis=1))[::-1][:11]
        top_10_words = np.array(vectorizer_tftidf.get_feature_names())[idx].tolist()
        top_10_words.remove('alice')
        print('Chapter {i}:'.format(i = i), top_10_words)

all_text()   


Chapter 1: ['wa', 'little', 'way', 'like', 'think', 'get', 'see', 'door', 'well', 'thought']
Chapter 2: ['wa', 'little', 'mouse', 'said', 'dear', 'pool', 'went', 'cat', 'foot', 'like']
Chapter 3: ['said', 'mouse', 'wa', 'know', 'dodo', 'lory', 'dry', 'one', 'prize', 'could']
Chapter 4: ['wa', 'little', 'said', 'rabbit', 'one', 'window', 'bill', 'heard', 'thought', 'sure']
Chapter 5: ['said', 'caterpillar', 'wa', 'serpent', 'pigeon', 'see', 'well', 'little', 'know', 'think']
Chapter 6: ['said', 'wa', 'cat', 'like', 'know', 'duchess', 'mad', 'little', 'much', 'footman']
Chapter 7: ['said', 'hatter', 'march', 'hare', 'dormouse', 'wa', 'know', 'time', 'well', 'thing']
Chapter 8: ['said', 'queen', 'wa', 'head', 'king', 'like', 'cat', 'three', 'went', 'one']
Chapter 9: ['said', 'turtle', 'mock', 'wa', 'gryphon', 'duchess', 'went', 'queen', 'little', 'never']
Chapter 10: ['said', 'gryphon', 'turtle', 'mock', 'lobster', 'beautiful', 'soup', 'soooop', 'would', 'whiting']
Chapter 11: ['said', 'w

In [4]:
data = pd.read_csv('Text.txt', delimiter="\t")
data.iloc[:,0] = data.iloc[:,0].apply(lambda x: clean_text(x))
X = data.iloc[:,0].tolist()
X = [X[i].split() for i in range(len(X))]
verbs = set(verbs)
from gensim.models import Word2Vec
model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=5)

In [5]:
model.build_vocab(X, progress_per=10000)


In [6]:
model.train(X, total_examples=model.corpus_count, epochs=30, report_delay=1)

(14325, 390300)

In [7]:
print(model.corpus_count)

2475


In [9]:
ans = model.wv.most_similar(positive=["alice"], topn = 40)
for obj in ans: 
    if (obj[0] in verbs):
        print(obj)

('said', 0.9946261644363403)
('thought', 0.9899784326553345)
('began', 0.9878082275390625)
('sure', 0.98622065782547)
('went', 0.9853482246398926)
('know', 0.9842615723609924)
('get', 0.9839959144592285)
('replied', 0.9830548763275146)
('like', 0.9820861220359802)
('got', 0.9808074235916138)
('came', 0.9799314737319946)
('dear', 0.9793164730072021)
('looked', 0.9778375625610352)
('getting', 0.9769870042800903)
('come', 0.9764481782913208)
('say', 0.9763820767402649)
('next', 0.9757989048957825)


Thus, the most common verbs are: ['say', 'thought', 'began', 'like', 'go', 'get', 'know', 'reply', 'look', 'came']